In [ ]:
import os
os.chdir('dummy_to_replace_with_blood_aging_dir_path/code')
# print(os.getcwd())


import sys
import shutil
# caution: path[0] is reserved for script path (or '' in REPL)

import importlib
import itertools
import metacells as mc
import anndata as ad
import pandas as pd
import numpy as np
import scipy.cluster.hierarchy
import scipy.spatial.distance
import matplotlib
import seaborn as sb
import matplotlib.pyplot as plt
import pathlib
import logging
import re
import collections
import sklearn
import sklearn.compose
import pickle

# from generic import gene_module_utils
from generic import generic_utils
from generic import ml_utils
from mds import mds_analysis_params
from mds import mds_analysis
# from mds import ipssm_utils
from generic import mc_utils
from mds import arch_mutation_interface_and_utils
from mds import clinical_data_interface_and_utils
from sc_rna_seq_preprocessing import sc_rna_seq_preprocessing_params

plt.rcParams["patch.force_edgecolor"] = False
plt.rcParams['patch.linewidth'] = 0
plt.rcParams['patch.edgecolor'] = 'none'
# plt.rcParams['scatter.edgecolors'] = 'black' # didnt affect sb.scatterplot

%matplotlib widget
%load_ext autoreload
%autoreload 2

def get_mds_params():
    return mds_analysis_params.MDS_ANALYSIS_PARAMS


def get_sc_rna_seq_preprocessing_params():
    return sc_rna_seq_preprocessing_params.SC_RNA_SEQ_PREPROCESSING_PARAMS

plt.subplots()
plt.close('all')

In [ ]:
all_misc_numbered_donor_info = mds_analysis.get_all_misc_numbered_donor_info()

xgboost_numbered_donor_df = pd.read_csv(get_mds_params()['all_numbered_donor_df_csv_file_path'])
xgboost_ext_donor_feature_df = pd.read_csv(get_mds_params()['all_ext_donor_feature_df_csv_file_path'])
xgboost_ext_donor_feature_df = xgboost_ext_donor_feature_df[xgboost_ext_donor_feature_df[['in_test_set', 'in_train_set']].any(axis=1)]

diagnosis_classes_of_interest = [
    'cytopenia',
    'MDS',
    'MDS/MPN',
    'normal',
]

xgboost_numbered_donor_df = generic_utils.merge_preserving_df1_index_and_row_order(xgboost_numbered_donor_df, xgboost_ext_donor_feature_df[['numbered_donor_id', 'in_test_set', 'in_train_set']], how='left')
xgboost_numbered_donor_df = xgboost_numbered_donor_df[xgboost_numbered_donor_df[['in_test_set', 'in_train_set']].any(axis=1)]


In [ ]:
max_cbc_10x_sample_days_dist = np.inf


possible_composition_cols = {f'log_c_{x}' for x in mds_analysis_params.PB_HSPC_STATE_NAMES}
c_composition_feature_cols = sorted(possible_composition_cols & set(xgboost_ext_donor_feature_df.columns))

gene_prog_sig_names = list(get_mds_params()['pb_sig_name_to_info'])
possible_gene_prog_cols = {f'{x}_{y}_0.5' for x, y in itertools.product(gene_prog_sig_names, mds_analysis_params.PB_HSPC_STATE_NAMES)}
median_gene_prog_cols = sorted(possible_gene_prog_cols & set(xgboost_ext_donor_feature_df.columns))
median_gene_prog_cols_without_hla = [x for x in median_gene_prog_cols if '_hla_sig_' not in x]
hla_gene_prog_cols = sorted(set(median_gene_prog_cols) - set(median_gene_prog_cols_without_hla))
print(f'hla_gene_prog_cols: {hla_gene_prog_cols}')
binary_snv_cols = [x for x in xgboost_ext_donor_feature_df.columns if x.endswith('_snv')]

possible_c_state_hsc_mpp_median_umi_count_log_ratio_cols = {f'c_{x}_HSC_MPP_median_umi_count_log_ratio' for x in mds_analysis_params.PB_HSPC_STATE_NAMES}
c_state_hsc_mpp_median_umi_count_log_ratio_cols = sorted(possible_c_state_hsc_mpp_median_umi_count_log_ratio_cols & set(xgboost_ext_donor_feature_df.columns))

cbc_cols = [
    'Basophils#', 'Basophils%', 'Eosinophils#', 'Eosinophils%', 'Hematocrit (%)', 'Hemoglobin (g/dl)', 'Lympho#', 'Lympho%', 'MCH (pg)', 'MCHC (g/dl)', 'MCV (fL)', 'MPV (fL)', 'Mono#', 'Mono%', 'Neutro#', 'Neutro%', 'Platelets (10^3/microliter)', 'RBC (10^6/microliter)', 'RDW (%)', 'WBC (10^3/microliter)',
]

composition_gene_progs_and_cna_count_cols = [
    *c_composition_feature_cols,
    *median_gene_prog_cols_without_hla,
    'cna_count',
]
final_features_without_age_and_sex_and_vaf_and_hla = [
    *cbc_cols,
    *c_composition_feature_cols,
    *median_gene_prog_cols_without_hla,
    'cna_count',
    'mean_near_neighbor_dist',
    
    # *c_state_hsc_mpp_median_umi_count_log_ratio_cols,
    # *mds_analysis_params.NEIGHBOR_STATE_LOG_RATIO_COLS,
    # 'log_c_clp_e',
]
final_features_without_age_and_sex_and_vaf = [
    *final_features_without_age_and_sex_and_vaf_and_hla,
    *hla_gene_prog_cols,
]
final_features_without_age_and_sex = [
    *final_features_without_age_and_sex_and_vaf,
    'max_mean_VAF',
]
final_features_with_age_and_sex_and_vaf = [
    *final_features_without_age_and_sex,
    'donor_age',
    'donor_sex',
]

feature_df_for_classifier = xgboost_ext_donor_feature_df[['numbered_donor_id', *final_features_with_age_and_sex_and_vaf]].copy()
cbc_too_far_from_10x_mask = xgboost_ext_donor_feature_df['cbc_10x_sample_days_dist'] > max_cbc_10x_sample_days_dist
feature_df_for_classifier.loc[cbc_too_far_from_10x_mask, cbc_cols] = np.nan

feature_df_for_classifier['donor_sex'] = feature_df_for_classifier['donor_sex'].astype(str)
assert feature_df_for_classifier['donor_sex'].isin(['male', 'female', 'nan']).all()

feature_df_for_classifier['is_female'] = (feature_df_for_classifier['donor_sex'] == 'female').astype(int)
feature_df_for_classifier['is_male'] = (feature_df_for_classifier['donor_sex'] == 'male').astype(int)
feature_df_for_classifier.drop(columns='donor_sex', inplace=True)

feature_df_for_classifier.set_index('numbered_donor_id', inplace=True)
feature_df_for_classifier.index.name = None
print(f'final feature_df_for_classifier.shape: {feature_df_for_classifier.shape}')
feature_df_for_classifier = feature_df_for_classifier.T


final_feature_names_and_types_df = pd.DataFrame([
    *[{'name': x, 'type': 'CBC'} for x in cbc_cols],
    *[{'name': x, 'type': 'gene_prog_excluding_mhc_ii'} for x in median_gene_prog_cols_without_hla],
    *[{'name': x, 'type': 'mhc_ii_gene_prog'} for x in hla_gene_prog_cols],
    *[{'name': x, 'type': 'log_c_state_freq'} for x in c_composition_feature_cols],
    {'name': 'max_mean_VAF', 'type': 'DNA'},
    {'name': 'donor_age', 'type': 'demographic'},
    {'name': 'donor_sex', 'type': 'demographic'},
    {'name': 'mean_near_neighbor_dist', 'type': 'mean_near_neighbor_dist'},
    {'name': 'log_c_clp_e', 'type': 'log_c_clp_e'},
    {'name': 'cna_count', 'type': 'cna_count'},
])
final_feature_names_and_types_df.to_csv(get_mds_params()['final_feature_names_and_types_df_csv_file_path'], index=False)

with_vaf_classifier_final_feature_names_and_types_df = final_feature_names_and_types_df[
    final_feature_names_and_types_df['name'].isin(final_features_without_age_and_sex)].reset_index(drop=True)
assert len(with_vaf_classifier_final_feature_names_and_types_df) == len(final_features_without_age_and_sex)
with_vaf_classifier_final_feature_names_and_types_df.to_csv(get_mds_params()['with_vaf_classifier_final_feature_names_and_types_df_csv_file_path'], index=False)

In [ ]:
final_features_with_age_and_sex_and_vaf

In [ ]:
with_vaf_feature_type_count_df = final_feature_names_and_types_df[final_feature_names_and_types_df['name'].isin(final_features_without_age_and_sex)]['type'].value_counts()
print(f'with_vaf_feature_type_count_df:\n{with_vaf_feature_type_count_df}')

In [ ]:
import xgboost as xgb
xgb.__version__

In [ ]:
import shap
shap.__version__

In [ ]:
any_na_df = generic_utils.merge_preserving_df1_index_and_row_order(feature_df_for_classifier.loc[final_features_without_age_and_sex].isna().any(axis=1).reset_index(name='any_na').rename(columns={'index': 'name'}), final_feature_names_and_types_df)
feature_types_with_any_na = sorted(any_na_df[any_na_df['any_na']]['type'].unique())
print(f'feature_types_with_any_na: {feature_types_with_any_na}')
# any_na_df[['type', 'any_na']].value_counts()

In [ ]:
out_dir_path = 'temp/_xgboost_out'
pathlib.Path(out_dir_path).mkdir(parents=True, exist_ok=True)

all_gene_indices = []

skip_cv = True
skip_cv = False

exclude_final_select = True
exclude_final_select = False

hardcoded_gene_pool_feature_pval_df_csv_file_path = None

select_features_by, gene_to_pool_fdr_alpha, final_feature_selection_fdr_alpha = ('mds_vs_cyto', 0.05, 0.05)
select_features_by, gene_to_pool_fdr_alpha, final_feature_selection_fdr_alpha = ('all', 0.05, 0.1)
select_features_by, gene_to_pool_fdr_alpha, final_feature_selection_fdr_alpha = ('all', 0.05, 0.05)
select_features_by, gene_to_pool_fdr_alpha, final_feature_selection_fdr_alpha = ('mds_vs_cyto', 0.05, 0.2)
select_features_by, gene_to_pool_fdr_alpha, final_feature_selection_fdr_alpha = ('all', 0.05, 0.2)

select_features_by, gene_to_pool_fdr_alpha, final_feature_selection_fdr_alpha = ('all', 0.05, np.inf)
select_features_by, gene_to_pool_fdr_alpha, final_feature_selection_fdr_alpha = ('mds_vs_cyto', 0.05, 0.1)

    
no_pooled_genes = False
no_pooled_genes = True

no_pooled_genes_repr = '_no_DEGs' if no_pooled_genes else ''

if hardcoded_gene_pool_feature_pval_df_csv_file_path:
    pool_feature_pval_df = pd.read_csv(hardcoded_gene_pool_feature_pval_df_csv_file_path)
    if 0:
        plt.close('all')
        sb.histplot(pool_feature_pval_df['cv_count'], bins=20)
        raise
    pool_feature_pval_df = pool_feature_pval_df[pool_feature_pval_df['cv_count'] > 50]
    hardcoded_up_genes = list(pool_feature_pval_df.loc[pool_feature_pval_df['diff_type'] == 'up', 'feature'])
    hardcoded_down_genes = list(pool_feature_pval_df.loc[pool_feature_pval_df['diff_type'] == 'down', 'feature'])
    assert not (set(hardcoded_up_genes) & set(hardcoded_down_genes))

print('better not use age and sex because our cohort is biased for them')
plt.close('all')
illu_ult_numbered_donor_id_to_is_mds_or_mds_mpn_as_float = {
    k: (1.0 if (v in (
        'MDS', 
        'MDS/MPN',
    )) else 0)
    for k, v in all_misc_numbered_donor_info['numbered_donor_id_to_diagnosis_class'].items()
}

gene_state_reprs = [
    'CLP',
    'MEBEMP_L',
    'HSC_MPP',
    'ERYP',
    'BEMP', 
]

common_prediction_plot_kwargs = dict(
    name_to_hue=all_misc_numbered_donor_info['numbered_donor_id_to_diagnosis_class'], 
    palette=clinical_data_interface_and_utils.DIAGNOSIS_CLASS_TO_COLOR,
)

mds_mpn_nd_ids = list(xgboost_numbered_donor_df.loc[xgboost_numbered_donor_df['diagnosis_class'] == 'MDS/MPN', 'numbered_donor_id'])
mds_mds_mpn_nd_ids = list(xgboost_numbered_donor_df.loc[xgboost_numbered_donor_df['diagnosis_class'].isin(['MDS', 'MDS/MPN']), 'numbered_donor_id'])
mds_mds_mpn_cytopenia_nd_ids = list(xgboost_numbered_donor_df.loc[xgboost_numbered_donor_df['diagnosis_class'].isin(['MDS', 'MDS/MPN', 'cytopenia']), 'numbered_donor_id'])
non_mds_mds_mpn_nd_ids = list(xgboost_numbered_donor_df.loc[~xgboost_numbered_donor_df['diagnosis_class'].isin(['MDS', 'MDS/MPN']), 'numbered_donor_id'])

assert select_features_by in {'all', 'mds_vs_cyto'}
nd_ids_to_use_in_feature_selection = None if select_features_by == 'all' else mds_mds_mpn_cytopenia_nd_ids

# raise RuntimeError('why isnt N184 in test? because she has a tech rep! is in train.')
get_nd_id_to_col_val = lambda col: generic_utils.get_dict_mapping_one_df_column_to_other(feature_df_for_classifier, 'numbered_donor_id', col)

ult_nd_ids = list(xgboost_numbered_donor_df.loc[xgboost_numbered_donor_df['is_ultima'], 'numbered_donor_id'])

train_nd_ids = list(xgboost_numbered_donor_df.loc[xgboost_numbered_donor_df['in_train_set'], 'numbered_donor_id'])
test_nd_ids = list(xgboost_numbered_donor_df.loc[xgboost_numbered_donor_df['in_test_set'], 'numbered_donor_id'])

model_name_to_info = {
    f'predict_mds_with_vaf_select_by_{select_features_by}_final_fdr_{final_feature_selection_fdr_alpha}{no_pooled_genes_repr}': dict(
        curr_train_df=feature_df_for_classifier[train_nd_ids],
        # curr_test_df=feature_df_for_classifier[sorted(set(feature_df_for_classifier) & set(ult_nd_ids))],
        curr_test_df=feature_df_for_classifier[test_nd_ids],
        feature_col_subset=final_features_without_age_and_sex,
        # excluded_nd_ids=complex_karyo_nd_ids if exclude_complex_karyo else [], 
        nd_id_to_y=illu_ult_numbered_donor_id_to_is_mds_or_mds_mpn_as_float,
        nd_ids_to_use_in_feature_selection=nd_ids_to_use_in_feature_selection,
            
        # arg_name_and_vals_and_plot_log=('max_depth', range(2, 5), False),
        # arg_name_and_vals_and_plot_log=('n_estimators', range(2, 15, 1), False), 
        # arg_name_and_vals_and_plot_log=('gamma', np.arange(8, 10, 1), False), 
        # arg_name_and_vals_and_plot_log=('min_child_weight', np.logspace(0.3, 1, 20), True),
        # cv_kwargs=dict(
        #     n_estimators=10, # sklearn default is 100. native api default is 10, it seems?. see https://stackoverflow.com/questions/68803420/what-is-the-default-value-of-n-estimators-in-xgboost-model/68806086#68806086, https://github.com/dmlc/xgboost/blob/master/python-package/xgboost/sklearn.py, https://xgboost.readthedocs.io/en/latest/python/python_api.html#xgboost.train (num_boost_round)
        #     max_depth=2, # IIUC, the default is 6 (https://xgboost.readthedocs.io/en/stable/parameter.html)
        # ),
    ),
    f'predict_mds_without_vaf_select_by_{select_features_by}_final_fdr_{final_feature_selection_fdr_alpha}{no_pooled_genes_repr}': dict(
        curr_train_df=feature_df_for_classifier[train_nd_ids],
        # curr_test_df=feature_df_for_classifier[sorted(set(feature_df_for_classifier) & set(ult_nd_ids))],
        curr_test_df=feature_df_for_classifier[test_nd_ids],
        feature_col_subset=final_features_without_age_and_sex_and_vaf,
        # excluded_nd_ids=complex_karyo_nd_ids if exclude_complex_karyo else [], 
        nd_id_to_y=illu_ult_numbered_donor_id_to_is_mds_or_mds_mpn_as_float,
        nd_ids_to_use_in_feature_selection=nd_ids_to_use_in_feature_selection,
            
        # arg_name_and_vals_and_plot_log=('max_depth', range(3, 6), False), # almost no difference
        # arg_name_and_vals_and_plot_log=('n_estimators', range(5, 15, 2), False), # 13 is better than 5
        # arg_name_and_vals_and_plot_log=('gamma', np.arange(0, 11, 2), False), # 0 is best
        # arg_name_and_vals_and_plot_log=('min_child_weight', np.arange(0, 11, 2), False), # varies, but 0 and 8 are good...
        # arg_name_and_vals_and_plot_log=('min_child_weight', np.logspace(0.3, 1, 20), True),
        # cv_kwargs=dict(
        #     n_estimators=10, # sklearn default is 100. native api default is 10, it seems?. see https://stackoverflow.com/questions/68803420/what-is-the-default-value-of-n-estimators-in-xgboost-model/68806086#68806086, https://github.com/dmlc/xgboost/blob/master/python-package/xgboost/sklearn.py, https://xgboost.readthedocs.io/en/latest/python/python_api.html#xgboost.train (num_boost_round)
        #     max_depth=2, # IIUC, the default is 6 (https://xgboost.readthedocs.io/en/stable/parameter.html)
        # ),
    ),


    f'predict_mds_with_comp_and_gene_progs_and_cna_select_by_{select_features_by}_final_fdr_{final_feature_selection_fdr_alpha}{no_pooled_genes_repr}': dict(
        curr_train_df=feature_df_for_classifier[train_nd_ids],
        curr_test_df=feature_df_for_classifier[sorted(set(feature_df_for_classifier) & set(ult_nd_ids))],
        feature_col_subset=composition_gene_progs_and_cna_count_cols,
        # excluded_nd_ids=complex_karyo_nd_ids if exclude_complex_karyo else [], 
        nd_id_to_y=illu_ult_numbered_donor_id_to_is_mds_or_mds_mpn_as_float,
        nd_ids_to_use_in_feature_selection=nd_ids_to_use_in_feature_selection,
            
        # arg_name_and_vals_and_plot_log=('max_depth', range(2, 5), False),
        # arg_name_and_vals_and_plot_log=('n_estimators', range(2, 15, 1), False), 
        # arg_name_and_vals_and_plot_log=('gamma', np.arange(8, 10, 1), False), 
        # arg_name_and_vals_and_plot_log=('min_child_weight', np.logspace(0.3, 1, 20), True),
        # cv_kwargs=dict(
        #     n_estimators=10, # sklearn default is 100. native api default is 10, it seems?. see https://stackoverflow.com/questions/68803420/what-is-the-default-value-of-n-estimators-in-xgboost-model/68806086#68806086, https://github.com/dmlc/xgboost/blob/master/python-package/xgboost/sklearn.py, https://xgboost.readthedocs.io/en/latest/python/python_api.html#xgboost.train (num_boost_round)
        #     max_depth=2, # IIUC, the default is 6 (https://xgboost.readthedocs.io/en/stable/parameter.html)
        # ),
    ),
    f'predict_mds_with_vaf_without_hla_select_by_{select_features_by}_final_fdr_{final_feature_selection_fdr_alpha}{no_pooled_genes_repr}': dict(
        curr_train_df=feature_df_for_classifier[train_nd_ids],
        curr_test_df=feature_df_for_classifier[sorted(set(feature_df_for_classifier) & set(ult_nd_ids))],
        feature_col_subset=[*final_features_without_age_and_sex_and_vaf_and_hla, 'max_mean_VAF'],
        # excluded_nd_ids=complex_karyo_nd_ids if exclude_complex_karyo else [], 
        nd_id_to_y=illu_ult_numbered_donor_id_to_is_mds_or_mds_mpn_as_float,
        nd_ids_to_use_in_feature_selection=nd_ids_to_use_in_feature_selection,
            
        # arg_name_and_vals_and_plot_log=('max_depth', range(2, 5), False),
        # arg_name_and_vals_and_plot_log=('n_estimators', range(2, 15, 1), False), 
        # arg_name_and_vals_and_plot_log=('gamma', np.arange(8, 10, 1), False), 
        # arg_name_and_vals_and_plot_log=('min_child_weight', np.logspace(0.3, 1, 20), True),
        # cv_kwargs=dict(
        #     n_estimators=10, # sklearn default is 100. native api default is 10, it seems?. see https://stackoverflow.com/questions/68803420/what-is-the-default-value-of-n-estimators-in-xgboost-model/68806086#68806086, https://github.com/dmlc/xgboost/blob/master/python-package/xgboost/sklearn.py, https://xgboost.readthedocs.io/en/latest/python/python_api.html#xgboost.train (num_boost_round)
        #     max_depth=2, # IIUC, the default is 6 (https://xgboost.readthedocs.io/en/stable/parameter.html)
        # ),
    ),
    f'predict_mds_by_cbc_and_vaf_only_select_by_{select_features_by}_final_fdr_{final_feature_selection_fdr_alpha}{no_pooled_genes_repr}': dict(
        curr_train_df=feature_df_for_classifier[train_nd_ids],
        curr_test_df=feature_df_for_classifier[sorted(set(feature_df_for_classifier) & set(ult_nd_ids))],
        feature_col_subset=[*cbc_cols, 'max_mean_VAF'],
        # excluded_nd_ids=complex_karyo_nd_ids if exclude_complex_karyo else [], 
        nd_id_to_y=illu_ult_numbered_donor_id_to_is_mds_or_mds_mpn_as_float,
        nd_ids_to_use_in_feature_selection=nd_ids_to_use_in_feature_selection,
            
        # arg_name_and_vals_and_plot_log=('max_depth', range(2, 5), False),
        # arg_name_and_vals_and_plot_log=('n_estimators', range(2, 15, 1), False), 
        # arg_name_and_vals_and_plot_log=('gamma', np.arange(8, 10, 1), False), 
        # arg_name_and_vals_and_plot_log=('min_child_weight', np.logspace(0.3, 1, 20), True),
        # cv_kwargs=dict(
        #     n_estimators=10, # sklearn default is 100. native api default is 10, it seems?. see https://stackoverflow.com/questions/68803420/what-is-the-default-value-of-n-estimators-in-xgboost-model/68806086#68806086, https://github.com/dmlc/xgboost/blob/master/python-package/xgboost/sklearn.py, https://xgboost.readthedocs.io/en/latest/python/python_api.html#xgboost.train (num_boost_round)
        #     max_depth=2, # IIUC, the default is 6 (https://xgboost.readthedocs.io/en/stable/parameter.html)
        # ),
    ),
    f'predict_mds_by_cbc_only_select_by_{select_features_by}_final_fdr_{final_feature_selection_fdr_alpha}{no_pooled_genes_repr}': dict(
        curr_train_df=feature_df_for_classifier[train_nd_ids],
        curr_test_df=feature_df_for_classifier[sorted(set(feature_df_for_classifier) & set(ult_nd_ids))],
        feature_col_subset=cbc_cols,
        # excluded_nd_ids=complex_karyo_nd_ids if exclude_complex_karyo else [], 
        nd_id_to_y=illu_ult_numbered_donor_id_to_is_mds_or_mds_mpn_as_float,
        nd_ids_to_use_in_feature_selection=nd_ids_to_use_in_feature_selection,
            
        # arg_name_and_vals_and_plot_log=('max_depth', range(2, 5), False),
        # arg_name_and_vals_and_plot_log=('n_estimators', range(2, 15, 1), False), 
        # arg_name_and_vals_and_plot_log=('gamma', np.arange(8, 10, 1), False), 
        # arg_name_and_vals_and_plot_log=('min_child_weight', np.logspace(0.3, 1, 20), True),
        # cv_kwargs=dict(
        #     n_estimators=10, # sklearn default is 100. native api default is 10, it seems?. see https://stackoverflow.com/questions/68803420/what-is-the-default-value-of-n-estimators-in-xgboost-model/68806086#68806086, https://github.com/dmlc/xgboost/blob/master/python-package/xgboost/sklearn.py, https://xgboost.readthedocs.io/en/latest/python/python_api.html#xgboost.train (num_boost_round)
        #     max_depth=2, # IIUC, the default is 6 (https://xgboost.readthedocs.io/en/stable/parameter.html)
        # ),
    ),
    f'predict_mds_without_vaf_and_hla_select_by_{select_features_by}_final_fdr_{final_feature_selection_fdr_alpha}{no_pooled_genes_repr}': dict(
        curr_train_df=feature_df_for_classifier[train_nd_ids],
        curr_test_df=feature_df_for_classifier[sorted(set(feature_df_for_classifier) & set(ult_nd_ids))],
        feature_col_subset=final_features_without_age_and_sex_and_vaf_and_hla,
        # excluded_nd_ids=complex_karyo_nd_ids if exclude_complex_karyo else [], 
        nd_id_to_y=illu_ult_numbered_donor_id_to_is_mds_or_mds_mpn_as_float,
        nd_ids_to_use_in_feature_selection=nd_ids_to_use_in_feature_selection,
            
        # arg_name_and_vals_and_plot_log=('max_depth', range(3, 6), False), # almost no difference
        # arg_name_and_vals_and_plot_log=('n_estimators', range(5, 15, 2), False), # 13 is better than 5
        # arg_name_and_vals_and_plot_log=('gamma', np.arange(0, 11, 2), False), # 0 is best
        # arg_name_and_vals_and_plot_log=('min_child_weight', np.arange(0, 11, 2), False), # varies, but 0 and 8 are good...
        # arg_name_and_vals_and_plot_log=('min_child_weight', np.logspace(0.3, 1, 20), True),
        # cv_kwargs=dict(
        #     n_estimators=10, # sklearn default is 100. native api default is 10, it seems?. see https://stackoverflow.com/questions/68803420/what-is-the-default-value-of-n-estimators-in-xgboost-model/68806086#68806086, https://github.com/dmlc/xgboost/blob/master/python-package/xgboost/sklearn.py, https://xgboost.readthedocs.io/en/latest/python/python_api.html#xgboost.train (num_boost_round)
        #     max_depth=2, # IIUC, the default is 6 (https://xgboost.readthedocs.io/en/stable/parameter.html)
        # ),
    ),
}

model_name_to_cv_res = {}
for model_name in [
    f'predict_mds_with_vaf_select_by_{select_features_by}_final_fdr_{final_feature_selection_fdr_alpha}{no_pooled_genes_repr}',
    f'predict_mds_without_vaf_select_by_{select_features_by}_final_fdr_{final_feature_selection_fdr_alpha}{no_pooled_genes_repr}',

    # f'predict_mds_with_comp_and_gene_progs_and_cna_select_by_{select_features_by}_final_fdr_{final_feature_selection_fdr_alpha}{no_pooled_genes_repr}',
    # f'predict_mds_with_vaf_without_hla_select_by_{select_features_by}_final_fdr_{final_feature_selection_fdr_alpha}{no_pooled_genes_repr}',
    # f'predict_mds_by_cbc_and_vaf_only_select_by_{select_features_by}_final_fdr_{final_feature_selection_fdr_alpha}{no_pooled_genes_repr}',
    # f'predict_mds_by_cbc_only_select_by_{select_features_by}_final_fdr_{final_feature_selection_fdr_alpha}{no_pooled_genes_repr}',
    # f'predict_mds_without_vaf_and_hla_select_by_{select_features_by}_final_fdr_{final_feature_selection_fdr_alpha}{no_pooled_genes_repr}',
]:
    model_info = model_name_to_info[model_name]
    curr_train_df = model_info['curr_train_df']
    curr_test_df = model_info['curr_test_df']
    nd_id_to_y = model_info['nd_id_to_y']

    is_classif = model_info.get('is_classif', True)
    random_state = model_info.get('random_state', 0)
    feature_col_subset = model_info.get('feature_col_subset', None)
    nd_ids_to_use_in_feature_selection = model_info.get('nd_ids_to_use_in_feature_selection', None)
    excluded_nd_ids = model_info.get('excluded_nd_ids', [])
    arg_name_and_vals_and_plot_log = model_info.get('arg_name_and_vals_and_plot_log', ('dummy', [0], False))
    cv_kwargs = model_info.get('cv_kwargs', {})
        
    if curr_test_df is not None:
        train_donor_ids = set(xgboost_numbered_donor_df.loc[xgboost_numbered_donor_df['numbered_donor_id'].isin(curr_train_df.columns), 'donor_id'])
        test_donor_ids = set(xgboost_numbered_donor_df.loc[xgboost_numbered_donor_df['numbered_donor_id'].isin(curr_test_df.columns), 'donor_id'])
        common_donor_ids = train_donor_ids & test_donor_ids
    
        assert (curr_test_df.index == curr_train_df.index).all()
    assert not common_donor_ids, f'train and test common_donor_ids: {common_donor_ids}'

    
    
    curr_train_df = curr_train_df[curr_train_df.columns[~curr_train_df.columns.isin(excluded_nd_ids)]]
    if curr_test_df is not None:
        curr_test_df = curr_test_df[curr_test_df.columns[~curr_test_df.columns.isin(excluded_nd_ids)]]
    if feature_col_subset is not None:
        curr_train_df = curr_train_df.loc[feature_col_subset]
        if curr_test_df is not None:
            curr_test_df = curr_test_df.loc[feature_col_subset]

    print(model_name, f'len(curr_train_df.columns): {len(curr_train_df.columns)}')
    if curr_test_df is not None:
        print(f'len(curr_test_df.columns): {len(curr_test_df.columns)}')
    # print(f'feature_cols: {curr_train_df.index}')

    nd_ids_to_use_in_feature_selection_mask = np.array(curr_train_df.columns.isin(nd_ids_to_use_in_feature_selection)) if (
        nd_ids_to_use_in_feature_selection is not None) else None

    train_target_vec = np.array([nd_id_to_y[x] for x in curr_train_df.columns])
    # train_target_vec = np.random.permutation(np.array([nd_id_to_y[x] for x in curr_train_df.columns]))
    train_target_nan_mask = np.isnan(train_target_vec)
    nan_train_target_count = train_target_nan_mask.sum()
    if nan_train_target_count > 0:
        print(f'discarding ({nan_train_target_count}) train samples with nan target ({curr_train_df.columns[train_target_nan_mask]})')
        curr_train_df = curr_train_df.loc[:, ~train_target_nan_mask]
        train_target_vec = train_target_vec[~train_target_nan_mask]

    if curr_test_df is None:
        test_target_vec = None
    else:
        test_target_vec = np.array([nd_id_to_y[x] for x in curr_test_df.columns])
        test_target_nan_mask = np.isnan(test_target_vec)
        nan_test_target_count = test_target_nan_mask.sum()
        if nan_test_target_count > 0:
            print(f'discarding ({nan_test_target_count}) test samples with nan target ({curr_test_df.columns[test_target_nan_mask]})')
            curr_test_df = curr_test_df.loc[:, ~test_target_nan_mask]
            test_target_vec = test_target_vec[~test_target_nan_mask]

    score_train_subgroup_name_to_mask = {}
    score_train_subgroup_name_to_mask = {'mds_cytopenia_only': np.array(curr_train_df.columns.isin(mds_mds_mpn_cytopenia_nd_ids))}
    score_train_subgroup_name_for_best_cv = 'all'
    score_train_subgroup_name_for_best_cv = 'mds_cytopenia_only'
    
    if hardcoded_gene_pool_feature_pval_df_csv_file_path:
        def curr_get_col_scores_and_masks_func(x, y, **kwargs):
            return np.full(x.shape[1], np.nan), [x.columns.isin(hardcoded_up_genes), x.columns.isin(hardcoded_down_genes)]
    else:
        def curr_get_col_scores_and_masks_func(x, y, **kwargs):
            return ml_utils.get_y_pos_and_neg_associated_feature_masks(x, y, fdr_alpha=gene_to_pool_fdr_alpha, ignore_nans=True, **kwargs)
    
    def curr_modify_pipe_inside_cv(pipe, train_mask):
        if nd_ids_to_use_in_feature_selection_mask is not None:
            train_sample_mask = nd_ids_to_use_in_feature_selection_mask[train_mask]
            
            pipe['select'].set_params(get_scores_and_support_mask_func_kwargs=dict(sample_mask=train_sample_mask))
            for t in pipe['select_and_pool_diff_exp_genes'].transformers:
                if not isinstance(t[1], ml_utils.PassthroughTransformer):
                    t[1].set_params(get_col_scores_and_masks_func_kwargs=dict(sample_mask=train_sample_mask))

    def xgboost_get_pipe(clf, exclude_final_select=False):
        col_transformer = sklearn.compose.ColumnTransformer(
            [(f'{st}_pooled_diff_genes', ml_utils.poolColsTransformer(['up', 'down'], get_col_scores_and_masks_func=curr_get_col_scores_and_masks_func), curr_train_df.index.isin([g for g in all_gene_indices if g.endswith(f'_{st}')])) for st in gene_state_reprs] + [
                (x, ml_utils.PassthroughTransformer(['']), curr_train_df.index == x) for x in curr_train_df.index[~curr_train_df.index.isin(all_gene_indices)]
            ],
        )
        col_transformer.set_output(transform='pandas')
        final_select_as_list = [] if exclude_final_select else [
            ('select', ml_utils.SelectorAllowingNans(lambda x, y, **kwargs: ml_utils.get_y_associated_feature_mask(x, y, fdr_alpha=final_feature_selection_fdr_alpha, ignore_nans=True, also_return_corrected_pvals=True, **kwargs)))
        ]
        pipe = sklearn.pipeline.Pipeline(steps=[
            ('select_and_pool_diff_exp_genes', col_transformer),
            *final_select_as_list,
            ('clf', clf),
        ])
        return pipe

    xgboost_cv_kwargs = dict(
        is_classif=is_classif,
        train_feature_df=curr_train_df.T,
        train_target_vec=train_target_vec,
        
        tree_method="hist", 
        # early_stopping_rounds=5,
        # min_child_weight=1, # default
        # eval_metric='logloss',
        arg_name_and_vals_and_plot_log=arg_name_and_vals_and_plot_log,
        score_train_subgroup_name_to_mask=score_train_subgroup_name_to_mask,
        score_train_subgroup_name_for_best_cv=score_train_subgroup_name_for_best_cv,
        
        random_state=(random_state + 123),
        seed=0,
        nthread=1,
        cv=sklearn.model_selection.LeaveOneOut(),
        title_prefix_for_plots=f'{model_name}, ',
        
        test_feature_df=None if curr_test_df is None else curr_test_df.loc[curr_train_df.index].T,
        test_target_vec=None if curr_test_df is None else np.array([nd_id_to_y[x] for x in curr_test_df.columns]),
        prediction_barplot_kwargs=dict(
            **common_prediction_plot_kwargs,
            show_prediction_plus1=True,
        ),
        prediction_scatterplot_kwargs=dict(
            **common_prediction_plot_kwargs,
            show_names=True,
        ),
        get_pipe=lambda clf: xgboost_get_pipe(clf, exclude_final_select=exclude_final_select),
        skip_cv=skip_cv,
        out_dir_path=out_dir_path,
        **cv_kwargs,
    )
    if hardcoded_gene_pool_feature_pval_df_csv_file_path and (not no_pooled_genes):
        xgboost_cv_kwargs['title_prefix_for_plots'] += 'hardcoded pooled genes, '
    if is_classif:
        xgboost_cv_kwargs = {
            **xgboost_cv_kwargs,
            **dict(
                objective='binary:logistic',
                modify_pipe_inside_cv_func=curr_modify_pipe_inside_cv,
            )
        }
    cv_res = ml_utils.xgboost_cv(**xgboost_cv_kwargs)
    # print('auc:', cv_res)

    model_name_to_cv_res[model_name] = cv_res
    cv_res['model_name'] = model_name
    
all_cv_res_df = pd.concat([y['cv_res_df'].assign(model_name=x) for x, y in model_name_to_cv_res.items()], ignore_index=True)
all_cv_res_df

In [ ]:
def get_mds_and_cytopenia_df(df):
    return df[df['name'].isin(mds_mds_mpn_cytopenia_nd_ids)]

for model_name, cv_res in model_name_to_cv_res.items():
    plt.close('all')
    fig, ax = ml_utils.plot_auc({
        'train': get_mds_and_cytopenia_df(cv_res['train_prediction_df']), 
        # 'train': cv_res['train_prediction_df'], 
        'test': get_mds_and_cytopenia_df(cv_res['test_prediction_df']),
        # 'test': cv_res['test_prediction_df'],
    }, title_suffix=model_name)
    # break
    without_vaf = model_name == 'predict_mds_without_vaf_select_by_mds_vs_cyto_final_fdr_0.1_no_DEGs'
    edf_or_fig_name = 'EDF_11C_' if without_vaf else 'fig_4F_'
    out_dir_path = 'temp/_fig_sup_4' if without_vaf else 'temp/_fig4'
    pathlib.Path(out_dir_path).mkdir(parents=True, exist_ok=True)
    fig.savefig(os.path.join(out_dir_path, f'{edf_or_fig_name}{model_name}_train_and_test_auc_curves.png'), dpi=300)